In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import mysql.connector

# 1) Configuration de google sheet

champ_application = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
References = ServiceAccountCredentials.from_json_keyfile_name("Credentials.json", champ_application)
client = gspread.authorize(References)

# Ouverture du classeur
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/10-BCkQZltFQWibW4gFz9Wfj8dED4-hEVjwcBNRf7FKM/edit?usp=sharing")
worksheet = spreadsheet.worksheet('Réponses au formulaire 3')

# Lire les données
records = worksheet.get_all_records()
df = pd.DataFrame(records)


df

,Horodateur,prénom,nom,téléphone1,téléphone2,email,diplome,sexe,age,profession,...,region_origine,membership,programme_intervention,En quelle qualité êtes-vous intervenus ?,année_intervention,Quelle thématique avez-vous abordé?,programme,année,pole,Souhaitez-vous figurer dans l'annuaire The Okwelians ?\nWould you like to be listed in The Okwelians' directory ?
0,21/02/2024 10:33:39,Dorine,MBONGALLE,237690308615,,dmbongalle@gmail.com,Post graduate dimplomat,Une femme,31-35 ans/ 31-35 years,Banker,...,Sud-Ouest / South-West,Membre de la family/ Member of The Family,,,,,,,,
1,21/02/2024 10:48:26,apolline,keou,237651514392,,apollinekeou@yahoo.fr,MASTER 2 EN MANAGEMENT - Master 2 en ingenierie,Une femme,Supérieur à 35 ans/ Above 35 years,Consultant en management et organisation,...,Ouest / West,Membre de la family/ Member of The Family,,,,,,,Équipe Exécutive / Executif team,
2,22/02/2024 17:20:38,Daïrou,Faridatou,693738385,,dairoufarida@gmail.com,Master 1,Une femme,18-25 ans/ 18-25 years,Étudiante,...,Nord / North,Lauréat.e/ Laureate,,,,,O'100,2022,Engagement/ Engagement,
3,25/02/2024 22:26:03,Hervé,Djia,237675858914,,djiathink@gmail.com,Master II Computer Science,Un homme,Supérieur à 35 ans/ Above 35 years,Consultant Système d’information,...,Ouest / West,Membre de la family/ Member of The Family,,,,,,,,
4,25/02/2024 22:27:48,Derrick,DANG A GOUFAN,675368159,,derrickdang15@yahoo.fr,Doctorat,Un homme,31-35 ans/ 31-35 years,Enseignant Chercheur,...,Centre / Center,Lauréat.e/ Laureate,,,,,CC (Campus chercheur),2022,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,12/05/2025 10:39:02,Elisabeth,SIMBE,237698763972,,Loveandcompassion1@yahoo.com,"BEPC / GCE ""O"" Level",Une femme / A woman,13/04/1979,Sans emploi /,...,Littoral / Littoral,Participant aux ateliers / workshop participants,,,,,,,Communication et Marketing/ Communication and ...,Oui
843,12/05/2025 10:49:11,Winner,NOA,+237 691758641,,noawinner1@gmail.com,BTS / Advanced Technician's Certificate,Un homme / A man,2005,Consultant,...,Centre / Center,Participant aux ateliers / workshop participants,,,,,,,,Oui
844,12/05/2025 13:30:12,Francis,FOTUE NGUEMNE,237696869682,237675691877,Betterfuture099@gmail.com,BTS / Advanced Technician's Certificate,Un homme / A man,16/01/1983,Journaliste / Journalist,...,Ouest / West,Participant aux ateliers / workshop participants,,,,,,,Stratégie et Organisation/ Strategy and Organi...,Oui
845,12/05/2025 14:25:10,LINDA CALVINE,NGANSOP,681836962,696068189,calvinengansop260@gmail.com,"BEPC / GCE ""O"" Level",Une femme / A woman,11/10/1994,Mobilisatrice communautaire,...,Ouest / West,Participant aux ateliers / workshop participants,,,,,,,Engagement/ Engagement,Oui


In [2]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()

# 4) Selectionner les données
Lauréat = df[df['membership'] == 'Lauréat.e/ Laureate']
Elders = df[df['membership'] == 'Ainé / Elder']
Partenaire = df[df['membership'] == 'Partenaire/ Partner']
Family = df[df['membership'] == 'Membre de la family/ Member of The Family']
Adhérent = df[df['membership'] == 'Membre adhérent "membre cotisant"/ Subscribing member "contributing member"']
Mentor= df[df['membership'] == 'Lauréat.e/ Laureate']
Autres= df[df['membership'] == 'Participant aux ateliers / workshop participants']

 #✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM lauréats")

#-- Vérifiez si l'email existe
for _, row in Lauréat.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM Lauréats WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO lauréats (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, programme, année_programme, année) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['programme'], row['année'], row['Horodateur'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

l'email supprimé: derrickdang15@yahoo.fr
l'email supprimé: natachamomo10@gmail.com
l'email supprimé: unkouefouette@gmail.com
l'email supprimé: wafoghislain2@gmail.com
l'email supprimé: armeladelaidensegue@gmail.com
l'email supprimé: alfredekango10@gmail.com
l'email supprimé: sommeleandelson@gmail.com
l'email supprimé: axeltagnef@gmail.com
l'email supprimé: mmakat71@gmail.com
l'email supprimé: amadoulabarang40@gmail.com
l'email supprimé: salecharlesjr@gmail.com
l'email supprimé: arolkache@gmail.com
l'email supprimé: ernestinejoynyangono@gmail.com
l'email supprimé: tsatedemstella@gmail.com
l'email supprimé: demsoungywetraissa@gmail.com
✅ Synchronisation terminée avec succès !


In [3]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM membre_de_la_family")

#-- Vérifiez si l'email existe
for _, row in Family.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM membre_de_la_family WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO membre_de_la_family (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

l'email supprimé: inkoukam@gmail.com
l'email supprimé: belingdominique@gmail.com
l'email supprimé: tmicaremedurance@gmail.com
✅ Synchronisation terminée avec succès !


In [4]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM membre_adhérents")

#-- Vérifiez si l'email existe
for _, row in Adhérent.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM membre_adhérents WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO membre_adhérents (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

✅ Synchronisation terminée avec succès !


In [5]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM partenaires")

#-- Vérifiez si l'email existe
for _, row in Partenaire.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM partenaires WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO partenaires (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

✅ Synchronisation terminée avec succès !


In [6]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM elders")

#-- Vérifiez si l'email existe
for _, row in Elders.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM elders WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO elders (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

✅ Synchronisation terminée avec succès !


In [7]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM autres")

#-- Vérifiez si l'email existe
for _, row in Autres.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM autres WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO autres (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année, membership) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'], row['membership'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

l'email supprimé: kengnineslinedjo@gmail.com
✅ Synchronisation terminée avec succès !


In [9]:
# ✅ 3) Connexion MySQL
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Annuaire",
  database="the_okwelians"
)
cursor = conn.cursor()


#✅ 4) Vider la table avant d'insérer (optionnel)
cursor.execute("DELETE FROM general")

#-- Vérifiez si l'email existe
for _, row in df.iterrows():
    email = row['email']
    doublon_query = "SELECT COUNT(*) FROM general WHERE email = %s"
    cursor.execute(doublon_query, (email,))
    count = cursor.fetchone()[0]

    if count == 0 :

        sql = "INSERT INTO general (prénom, nom, téléphone1, téléphone2, email, sexe, age, diplôme, profession, structure, secteur, secteur_activité, pays_de_résidence, région_de_résidence, département_de_résidence, ville_de_résidence, région_origine, année, membership, pole_benevolat) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row['prénom'], row['nom'], row['téléphone1'], row['téléphone2'], row['email'], row['sexe'], row['age'], row['diplome'], row['profession'], row['structure'], row['secteur'], row['secteur_activite'], row['pays_de_residence'], row['region_de_residence'], row['departement_de_residence'], row['ville_de_residence'], row['region_origine'], row['Horodateur'], row['membership'], row['pole'])
        cursor.execute(sql, values)

    else : 

         print("l'email supprimé:", email)
         

conn.commit()

# ✅ 6) Fermer la connexion
cursor.close()
conn.close()

        


  
print("✅ Synchronisation terminée avec succès !")

l'email supprimé: derrickdang15@yahoo.fr
l'email supprimé: natachamomo10@gmail.com
l'email supprimé: unkouefouette@gmail.com
l'email supprimé: wafoghislain2@gmail.com
l'email supprimé: armeladelaidensegue@gmail.com
l'email supprimé: daawemh@gmail.com
l'email supprimé: alfredekango10@gmail.com
l'email supprimé: sommeleandelson@gmail.com
l'email supprimé: axeltagnef@gmail.com
l'email supprimé: mmakat71@gmail.com
l'email supprimé: inkoukam@gmail.com
l'email supprimé: kengnineslinedjo@gmail.com
l'email supprimé: amadoulabarang40@gmail.com
l'email supprimé: mbone.joel@gmail.com
l'email supprimé: salecharlesjr@gmail.com
l'email supprimé: belingdominique@gmail.com
l'email supprimé: arolkache@gmail.com
l'email supprimé: tmicaremedurance@gmail.com
l'email supprimé: ernestinejoynyangono@gmail.com
l'email supprimé: olivierbjules237@gmail.com
l'email supprimé: celestintinabiyoo@yahoo.fr
l'email supprimé: fredyfokou7@gmail.com
l'email supprimé: Aurelientchako@gmail.com 
l'email supprimé: josapro3@g